# Análisis sobre el impacto del bitcoin en la macroeconomía de El Salvador

In [88]:
import pandas as pd
import os

ruta_absoluta = os.path.abspath('../Datos_bitcoin/Deuda_SV/Deuda_Pública_Total_Mensual(1).csv')
print(f'Ruta absoluta: {ruta_absoluta}')

df_deuda = pd.read_excel("C:\\Users\\zarfk\\Desktop\\Programacion\\SIC IA\\Proyecto_final\\Datos_bitcoin\\Deuda_SV\\Deuda_Publica_Total_Mensual.csv")
df_deuda.info()

Ruta absoluta: C:\Users\zarfk\Desktop\Programacion\SIC IA\Datos_bitcoin\Deuda_SV\Deuda_Pública_Total_Mensual(1).csv


ValueError: Excel file format cannot be determined, you must specify an engine manually.